## Homework: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27


In [1]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

c:\Users\kiramishima\.conda\envs\llm-zcamp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [17]:
v = embedding_model.encode(user_question)
v

array([ 7.82226622e-02, -4.01311517e-02,  3.86135913e-02, -1.78982707e-04,
        8.92347172e-02, -5.04591577e-02, -1.05026513e-02,  3.71055715e-02,
       -4.18714322e-02,  3.48084494e-02, -1.20701753e-02, -2.36942433e-02,
        3.87900174e-02,  1.60988811e-02,  3.50747406e-02,  3.04742320e-03,
        5.79672381e-02, -4.10627611e-02, -3.41552645e-02, -2.56396662e-02,
       -3.55263911e-02,  1.42907826e-02, -1.62799843e-02,  3.21446657e-02,
       -4.66897450e-02,  7.89185837e-02,  4.90161069e-02,  1.56761073e-02,
       -1.69109758e-02,  2.26482097e-02,  5.60206026e-02, -3.98361608e-02,
        6.77410290e-02, -1.20209698e-02,  1.12620974e-03, -1.94393992e-02,
       -2.65951566e-02,  1.06177507e-02,  1.69686973e-02,  1.13488063e-02,
       -2.97063086e-02,  5.25258295e-02, -1.41453864e-02,  4.61700261e-02,
        1.17065832e-02, -2.38053445e-02, -6.32558092e-02, -1.92041826e-02,
       -7.10591115e-03,  3.24168019e-02,  2.49618255e-02, -5.27501665e-03,
        2.01149154e-02, -

## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
filter_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        filter_docs.append(doc)

In [10]:
len(filter_docs)

375

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 

In [50]:
import numpy as np
from tqdm.notebook import tqdm
embeddings = []
embedded_documents = []

In [51]:
for doc in tqdm(filter_docs):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    doc["qa_embedding"] = embedding_model.encode(qa_text)
    embeddings.append(doc["qa_embedding"])
    embedded_documents.append(doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [15]:
X = np.array(embeddings)

In [16]:
X.shape

(375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return something very close to 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- 0.65
- 0.065

In [20]:
results = []
for emb in tqdm(embeddings):
    results.append(emb.dot(v))

  0%|          | 0/375 [00:00<?, ?it/s]

In [21]:
max(results)

0.65065724

## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

In [37]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filter_docs, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

In [23]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [72]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [75]:
filter_docs[0].keys()

dict_keys(['text', 'section', 'question', 'course', 'id', 'qa_embedding'])

In [25]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [48]:
def evaluate(ground_truth, search_engine):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        question = q['question']
        question_vector = embedding_model.encode(question)
        results = search_engine.search(question_vector, num_results=5)
        
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [49]:
evaluate(ground_truth, search_engine)


{'hit_rate': 0.9398907103825137}

## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [78]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'f70cc1e4d8cd', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rbCf9ni3S1qprRV69Lcp7w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [79]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "id": {"type": "keyword"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "qa_embedding": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity":
                "cosine"
            },
        }
    }
}

In [80]:
index_name = "course-questions-hw3"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw3'})

In [81]:
embedded_documents[0].keys()

dict_keys(['text', 'section', 'question', 'course', 'id', 'qa_embedding'])

In [82]:
for doc in tqdm(embedded_documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/375 [00:00<?, ?it/s]

In [83]:
es_query = {
    "field" : "qa_embedding",
    "query_vector" :  v,
    "k" : 5,
    "num_candidates" : 10000,
}

In [84]:
res = es_client.search(index=index_name, knn=es_query, source=["text","section","question","course"])
res["hits"]["hits"][0]["_id"]

'gpvNspABoFYOXg7pnNlp'

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

In [95]:
def elasticsearch_evaluate(ground_truth, index_name, es_client):
    cnt = 0

    for q in tqdm(ground_truth):
        question = q['question']
        document = q['document']
        question_vector = embedding_model.encode(question)
        # Query
        es_query = {
            "field" : "qa_embedding",
            "query_vector" : question_vector,
            "k" : 5,
            "num_candidates" : 10000,
        }
        result = es_client.search(index=index_name, knn=es_query, source=["text", "id", "section", "question", "course"])
        search_documents = [result['_source']['id'] for result in result["hits"]["hits"]]

        if document in search_documents:
            cnt += 1

    return {
        'hit_rate': cnt / len(ground_truth)
    }

In [96]:
elasticsearch_evaluate(ground_truth, index_name, es_client)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}